In [1]:
import numpy as np
import cv2
import time
import datetime
import pandas as pd
import pickle
from sklearn.cluster import KMeans

In [2]:
def draw_flow(img, flow, step=16):

    h, w = img.shape[:2]
    y, x = np.mgrid[step/2:h:step, step/2:w:step].reshape(2,-1).astype(int)
    fx, fy = flow[y,x].T

    lines = np.vstack([x, y, x-fx, y-fy]).T.reshape(-1, 2, 2)
    lines = np.int32(lines + 0.5)

    img_bgr = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    cv2.polylines(img_bgr, lines, 0, (0, 255, 0))

    for (x1, y1), (_x2, _y2) in lines:
        cv2.circle(img_bgr, (x1, y1), 1, (0, 255, 0), -1)

    return img_bgr


In [3]:
def draw_hsv(flow):

    h, w = flow.shape[:2]
    fx, fy = flow[:,:,0], flow[:,:,1]

    ang = np.arctan2(fy, fx) + np.pi
    v = np.sqrt(fx*fx+fy*fy)

    hsv = np.zeros((h, w, 3), np.uint8)
    hsv[...,0] = ang*(180/np.pi/2)
    hsv[...,1] = 255
    hsv[...,2] = np.minimum(v*4, 255)
    bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

    return bgr


In [8]:
from scipy.stats import mode

cap = cv2.VideoCapture("sample0.mp4")

suc, prev = cap.read()
prevgray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)
t1=datetime.datetime.now()

while True:
    start = time.time()                                           # start time to calculate FPS
    suc, img = cap.read()    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    #####################################################
    model = pickle.load(open('kmeansmodel.pkl', "rb"))
    orb = cv2.ORB_create()#initialise  detector
    keypoints, descriptors = orb.detectAndCompute(gray, None)
    out=pd.DataFrame(descriptors)    
    output=pd.DataFrame(model.predict(out))
    hist=np.histogram(output.values,bins=[0,1,2,3,4,5,6,7,8,9])
    histout=pd.DataFrame(hist[0])
    m=[]
    m.append(hist[0])
    loaded_model = pickle.load(open('finalized_model1.sav', 'rb'))
    output=pd.DataFrame(m)
    x1 = output.iloc[:, 1:]
    prediction=loaded_model.predict(x1)
    print(prediction)
        
    ####################################################

    
    
    flow = cv2.calcOpticalFlowFarneback(prevgray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    #print(flow)
    mag, ang = cv2.cartToPolar(flow[:, :, 0], flow[:, :, 1], angleInDegrees=True)
    move_sense = ang[mag > 10]
    move_mode = mode(move_sense)[0]
    print(move_mode)
    prevgray = gray
    # End time
    end = time.time()
    # calculate the FPS for current frame detection
    fps = 1 / (end-start)
    
    
    if((prediction== ['Sleeping'] or prediction==['Sitting']) and ( move_mode.size==0 )):
        t2=datetime.datetime.now()
        delta=t2-t1
        if(delta.seconds>=3): 
            print(str(t2)+" : ALERT")
            continue
    else: t1=datetime.datetime.now()
        
    print(f"{fps:.2f} FPS" )
    cv2.imshow('flow', draw_flow(gray, flow))
    cv2.resizeWindow("flow", 700,700)
    cv2.imshow('flow HSV', draw_hsv(flow))
    cv2.resizeWindow("flow HSV", 700, 700)


    key = cv2.waitKey(100)
    if key == ord('q'):
        break


cap.release()
cv2.destroyAllWindows()


['Standing']
[]
1.31 FPS
['Standing']
[]
4.76 FPS
['Standing']
[]
4.76 FPS
['Standing']
[]
4.79 FPS
['Standing']
[]
4.81 FPS
['Standing']
[]
5.24 FPS
['Standing']
[]
5.05 FPS
['Standing']
[]
5.38 FPS
['Standing']
[]
5.29 FPS
['Standing']
[]
5.08 FPS
['Standing']
[]
5.50 FPS
['Standing']
[]
5.32 FPS
['Standing']
[]
4.93 FPS
['Standing']
[]
5.50 FPS
['Standing']
[]
5.99 FPS
['Standing']
[]
5.59 FPS
['Standing']
[]
5.10 FPS
['Standing']
[]
5.27 FPS
['Standing']
[]
5.16 FPS
['Standing']
[]
5.47 FPS
['Standing']
[]
5.29 FPS
['Standing']
[]
5.00 FPS
['Standing']
[]
6.21 FPS
['Standing']
[]
5.44 FPS
['Standing']
[]
5.24 FPS
['Standing']
[]
5.29 FPS
['Standing']
[]
4.37 FPS
['Standing']
[]
5.24 FPS
['Standing']
[]
4.88 FPS
['Standing']
[]
4.81 FPS
['Standing']
[]
5.10 FPS
['Standing']
[]
5.10 FPS
['Standing']
[]
5.32 FPS
['Standing']
[]
4.70 FPS
['Standing']
[]
5.82 FPS
['Standing']
[]
4.29 FPS
['Standing']
[]
5.65 FPS
['Standing']
[]
5.35 FPS
['Standing']
[]
4.98 FPS
['Standing']
[]
5.35 FPS
